In [ ]:
import numpy as np
import pandas as pd
import os
import shutil

In [ ]:
FechaInicio = '20/10/2023'
FechaFinal = '20/11/2023'

In [ ]:
FechaComienzo = FechaInicio.replace('/', '-')
FechaTermino = FechaFinal.replace('/', '-')

In [ ]:
FileName_DataObservada = f"calidad-aire-dia-hora_Sierra_Gorda_{FechaComienzo}_a_{FechaTermino}.csv"

In [ ]:
FileName_DataModelada = f"calidad-aire-dia-hora_Sierra_Gorda-PRO_{FechaComienzo}_a_{FechaTermino}.csv"

In [ ]:
DataObservada = pd.read_csv(f'../Observados/ScrapedGeologger/{FileName_DataObservada}')

In [ ]:
DataModelada = pd.read_csv(f'../Modelados/ScrapedGeologger/{FileName_DataModelada}')

In [ ]:
# Add ":00" to 'Hora' to create a minute component
DataObservada['Hora'] = DataObservada['Hora'].astype(str) + ':00'

# Combine 'Fecha' and 'Hora' into 'Fecha_y_Hora'
DataObservada['Fecha_y_Hora'] = pd.to_datetime(DataObservada['Fecha'] + ' ' + DataObservada['Hora'], format='%d/%m/%Y %H:%M')

In [ ]:
# Add ":00" to 'Hora' to create a minute component
DataModelada['Hora'] = DataModelada['Hora'].astype(str) + ':00'

# Combine 'Fecha' and 'Hora' into 'Fecha_y_Hora'
DataModelada['Fecha_y_Hora'] = pd.to_datetime(DataModelada['Fecha'] + ' ' + DataModelada['Hora'], format='%d/%m/%Y %H:%M')

In [ ]:
# Merge the two dataframes on the datetime column
df_merged = pd.merge(DataObservada[['Fecha_y_Hora', 'MP10 (µg/m³N)']], DataModelada[['Fecha_y_Hora', 'MP10 (µg/m³N)']], on='Fecha_y_Hora', suffixes=('_obs', '_pro'))


In [ ]:
# Create 'Bueno' column
df_merged['Bueno'] = np.where(
    (((df_merged['MP10 (µg/m³N)_obs'] >= 0) & (df_merged['MP10 (µg/m³N)_obs'] <= 130) & 
      (df_merged['MP10 (µg/m³N)_pro'] >= 0) & (df_merged['MP10 (µg/m³N)_pro'] <= 130)) | 
     ((df_merged['MP10 (µg/m³N)_obs'] < 0) | (df_merged['MP10 (µg/m³N)_obs'] > 130) & 
      (df_merged['MP10 (µg/m³N)_pro'] < 0) | (df_merged['MP10 (µg/m³N)_pro'] > 130))), 1, 0)

# Create 'Alerta1' column
df_merged['Alerta1'] = np.where(
    (((df_merged['MP10 (µg/m³N)_obs'] > 130) & (df_merged['MP10 (µg/m³N)_obs'] <= 180) & 
      (df_merged['MP10 (µg/m³N)_pro'] > 130) & (df_merged['MP10 (µg/m³N)_pro'] <= 180)) | 
     ((df_merged['MP10 (µg/m³N)_obs'] <= 130) | (df_merged['MP10 (µg/m³N)_obs'] > 180) & 
      (df_merged['MP10 (µg/m³N)_pro'] <= 130) | (df_merged['MP10 (µg/m³N)_pro'] > 180))), 1, 0)

# Create 'Alerta2' column
df_merged['Alerta2'] = np.where(
    ((df_merged['MP10 (µg/m³N)_obs'] > 180) & (df_merged['MP10 (µg/m³N)_pro'] > 180)) | 
    ((df_merged['MP10 (µg/m³N)_obs'] < 180) & (df_merged['MP10 (µg/m³N)_pro'] < 180)), 1, 0)


In [ ]:
pct_bueno = df_merged['Bueno'].sum() / len(df_merged)
pct_alerta1 = df_merged['Alerta1'].sum() / len(df_merged)
pct_alerta2 = df_merged['Alerta2'].sum() / len(df_merged)

promedio = np.mean([pct_bueno, pct_alerta1, pct_alerta2])


In [ ]:
print(pct_bueno)
print(pct_alerta1)
print(pct_alerta2)

print(promedio)

0.9010416666666666
0.8828125
0.7552083333333334
0.8463541666666666
